In [1]:
import os
from google.colab import drive

# Монтируем Google Drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import zipfile
import pandas as pd

zip_path = '/content/drive/MyDrive/коменты хак/dataset.jsonl.zip'

extract_path = '/content/drive/MyDrive/коменты хак/'
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)


FileNotFoundError: ignored

In [3]:
import pandas as pd
data = pd.read_json('/content/drive/MyDrive/dataset.jsonl',lines=True)
df = pd.DataFrame(data)

LLM

In [4]:
import pandas as pd

# Assuming your DataFrame is named 'df'
# Split the DataFrame into batches (adjust batch size based on your requirements)
batch_size = 1000
batches = [df[i:i + batch_size] for i in range(0, df.shape[0], batch_size)]

# Create lists to store processed post and comment data
processed_posts = []
processed_comments = []

# Iterate through batches
for batch in batches:
    posts = batch[batch['root_id'].isna()]
    comments = batch[~batch['root_id'].isna()]

    # Preprocessing for posts
    processed_posts_batch = posts[['text', 'url', 'hash', 'id', 'date']].copy()

    # Preprocessing for comments
    processed_comments_batch = comments[['text', 'url', 'hash', 'id', 'date', 'root_id', 'parent_id']].copy()

    # Append processed batches to the lists
    processed_posts.append(processed_posts_batch)
    processed_comments.append(processed_comments_batch)

# Concatenate processed post and comment batches back into DataFrames
df_processed_posts = pd.concat(processed_posts, ignore_index=True)
df_processed_comments = pd.concat(processed_comments, ignore_index=True)

# Summarize all comments under each post with comment hashes separated by commas
all_comments = df_processed_comments.groupby('root_id')['text'].apply(lambda x: ' '.join(x)).reset_index()
all_comments_hash = df_processed_comments.groupby('root_id')['hash'].apply(lambda x: ', '.join(x)).reset_index()
all_comments_summary = pd.merge(all_comments, all_comments_hash, on='root_id', how='left')
all_comments_summary = pd.merge(all_comments_summary, df_processed_posts[['id', 'text']], left_on='root_id', right_on='id', how='left')
all_comments_summary = all_comments_summary.rename(columns={'id': 'post_id', 'text_x': 'comments', 'text_y': 'post_text', 'hash': 'comment_hashes'})
all_comments_summary = all_comments_summary[['post_id', 'comments', 'post_text', 'comment_hashes']]

# Summarize only comments explicitly related to the text of each post with comment hashes separated by commas
post_comments = df_processed_comments[~df_processed_comments['parent_id'].notna()]
post_comments_hash = df_processed_comments[~df_processed_comments['parent_id'].notna()].groupby('root_id')['hash'].apply(lambda x: ', '.join(x)).reset_index()
post_comments_summary = pd.merge(post_comments, post_comments_hash, on='root_id', how='left')
post_comments_summary = pd.merge(post_comments_summary, df_processed_posts[['id', 'text']], left_on='root_id', right_on='id', how='left')
post_comments_summary = post_comments_summary.rename(columns={'id': 'post_id', 'text_x': 'comments', 'text_y': 'post_text', 'hash': 'comment_hashes'})
#post_comments_summary = post_comments_summary[['post_id', 'comments', 'post_text', 'comment_hashes']]

# Summarize comments indirectly related to the post with comment hashes separated by commas
topic_comments = df_processed_comments[df_processed_comments['parent_id'].notna()]
topic_comments_hash = df_processed_comments[df_processed_comments['parent_id'].notna()].groupby('root_id')['hash'].apply(lambda x: ', '.join(x)).reset_index()
topic_comments_summary = pd.merge(topic_comments, topic_comments_hash, on='root_id', how='left')
topic_comments_summary = pd.merge(topic_comments_summary, df_processed_posts[['id', 'text']], left_on='root_id', right_on='id', how='left')
topic_comments_summary = topic_comments_summary.rename(columns={'id': 'post_id', 'text_x': 'comments', 'text_y': 'post_text', 'hash': 'comment_hashes'})
#topic_comments_summary = topic_comments_summary[['post_id', 'comments', 'post_text', 'comment_hashes']]

# Display the results
#print(all_comments_summary)
#print(post_comments_summary)
#print(topic_comments_summary)


<ipython-input-4-c2f289844147>:41: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  post_comments_hash = df_processed_comments[~df_processed_comments['parent_id'].notna()].groupby('root_id')['hash'].apply(lambda x: ', '.join(x)).reset_index()


In [21]:
topic_comments_summary=topic_comments_summary[:10]

In [22]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("DeepPavlov/mbart-large-50-ru-persona-chat")
model = AutoModelForSeq2SeqLM.from_pretrained("DeepPavlov/mbart-large-50-ru-persona-chat")
# Add padding token to tokenizer
tokenizer.add_special_tokens({'pad_token': '<pad>'})
model.resize_token_embeddings(len(tokenizer))

# Your DataFrame
# ...

# List to store related comments
related_comments = []

# Batch size for processing
batch_size = 16

# Process comments in batches
for i in range(0, len(topic_comments_summary), batch_size):
    batch_comments = topic_comments_summary['comments'].iloc[i:i+batch_size].tolist()
    batch_post_texts = topic_comments_summary['post_text'].iloc[i:i+batch_size].tolist()

    # Combine the prompt template with the comments and post texts
    prompts = [
        f"Контекст: Определите комментарии, прямо связанные с текстом поста.\n{c}\n\nТекст поста: {pt}\n\nВопрос: Сопоставьте комментарий с постом.\nОтвет: ..."
        for c, pt in zip(batch_comments, batch_post_texts)
    ]

    # Tokenize on GPU
    inputs = tokenizer(prompts, return_tensors="pt", truncation=True, padding=True)

    # Generate model outputs on GPU
    with torch.no_grad():
        logits = model(**inputs).logits.mean(dim=1)

  +++++++++++++++++   related_comments.extend([comment for comment, logit in zip(batch_comments, logits) if logit > 0.5])

# Create a new DataFrame with related comments
df_related_comments = pd.DataFrame({'related_comments': related_comments})

# Display the new DataFrame
print(df_related_comments)


KeyboardInterrupt: ignored

In [58]:
topic_comments_summary=topic_comments_summary[:500]

In [25]:
!pip install sentencepiece


In [6]:
!pip install rouge_score googletrans -qU

In [7]:
!pip install hugginface_hub
from huggingface_hub import notebook_login
notebook_login()

ERROR: Could not find a version that satisfies the requirement hugginface_hub (from versions: none)
ERROR: No matching distribution found for hugginface_hub


In [8]:
!pip install transformers

In [27]:
!pip install datsets transformers[sentencepiece]
!pip install sentencepiece

ERROR: Could not find a version that satisfies the requirement datsets (from versions: none)
ERROR: No matching distribution found for datsets


после установки библиотек нужно перезапустить среду выполнения)

In [ ]:
all_comments_summary

In [12]:
!pip install bert_score -qU

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 1.9 MB/s eta 0:00:00


In [9]:
all_comments_summary

,post_id,comments,post_text,comment_hashes
0,--SDWklGeAE,@kryxmks. Зная большую часть людей в коммент...,"Приколы,BITTUEV,юмор,вайны,лучшие приколы,кари...","bb746cbc009b15cd6a0f451b102cb31e, 4c8a31f1ef12..."
1,-100194451_510298,"[id50188945|Сергей], это разве Угол))). Чуть ч...",Штрафы за дрифт начали приходить 🙈<hr/><br/><a...,"7361155a282dda4629d018f07b4c24a9, 959aafd544be..."
2,-100272777_810169,Да нет надо просто с лопатами к Администрации ...,Анонимно пожалуйста куда ещё звонить или писат...,"e95970a83cee2788f8b968b4b6147236, edffa95bbb21..."
3,-100283479_678834,Кошель та с рунами 🤔 Смотря сколько взять... е...,"В магазине Пятерочка ул.Манченко, д.25 найден ...","4be8b6682fb38fbd172c0845fb78e1eb, 8438cd381364..."
4,-100283479_679838,"автор эстонец? [id22566180|Юрий], ""помню был у...","28.11.2023 , примерно в 20:30 вечера произошла...","a069cc8ada28697069556d997d194a3b, 0904907e4fcd..."
5,-100345006_751238,Ты подарил мне в день рождения стиральную маши...,#начало или #продолжи_фразу\nТы подарил мне в ...,"130ee0e3b90713eb23bdcdf17f589894, 4832129d64f2..."
6,-1003715_1457957,Значит ли это что мы не скоро увидим но...,Неизданные песни Майкла Джексона продают на ау...,"927e493be792985883ee93e74147435c, 1cccb105185e..."
7,-100426847_41979,Какой наполнитель? Цена? Ну нах такую красоту ...,Красотки 🥰Любимки наши ❤️<br>А на Северной 76 ...,"38ecc3d99768e2042b678afbd1dac877, bab6c5d6da43..."
8,-100548814_121765,"[id156309073|Александр], я не против, но ОЧЕНЬ...",🔥 Корейцы есть? 😄 Если у вас есть знакомые / р...,"0aaec9177e87a372d2f048012bf5d25c, d01e0225d0d2..."
9,-10055384_7436142,"[id91837649|Віктор], 3 вроде даже, одно беспла...",Одна из лучших глав в Alan Wake II чуть не пош...,"dfbb3da272473b9a41334bfdc0ab5d2c, f74acdf7dd41..."


In [18]:
all_comments_summary=all_comments_summary[0:30]

In [ ]:
!pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=5f381fcf82b2efd5519b3560fd38a6ad6aee63a613bd9cdcb8601a70b7b26b66
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [ ]:
all_comments_summary=all_comments_summary[:10]

In [27]:
import torch
from transformers import T5ForConditionalGeneration, T5Tokenizer
import pandas as pd
import time
from concurrent.futures import ThreadPoolExecutor

# Constants
MODEL_NAME = 'cointegrated/rut5-base-absum'
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Initializing T5 model and tokenizer
def initialize_model():
    model = T5ForConditionalGeneration.from_pretrained(MODEL_NAME).to(DEVICE)
    tokenizer = T5Tokenizer.from_pretrained(MODEL_NAME)
    return model, tokenizer

# Summarization function
def summarize(text, model, tokenizer, n_words=3, max_length=10000, num_beams=2, do_sample=False, repetition_penalty=10.0, **kwargs):
    if n_words:
        text = f'[{n_words}] {text}'
    inputs = tokenizer(text, return_tensors='pt', padding=True).to(DEVICE)

    with torch.no_grad():
        out = model.generate(
            **inputs,
            max_length=max_length,
            num_beams=num_beams,
            do_sample=do_sample,
            repetition_penalty=repetition_penalty,
            **kwargs
        )

    return tokenizer.decode(out[0], skip_special_tokens=True)

# Summarization for batches
def summarize_batch(batch, model, tokenizer, cache):
    results = []
    for text in batch:
        if text not in cache:
            summary = summarize(text, model, tokenizer)
            cache[text] = summary
        else:
            summary = cache[text]
        results.append(summary)

    # Clear GPU cache after processing each batch to release memory
    torch.cuda.empty_cache()

    return results

# Parallelized batch processing
def summarize_batch_parallel(comments, model, tokenizer, cache, batch_size=256):
    with ThreadPoolExecutor() as executor:
        batches = [comments[i:i + batch_size] for i in range(0, len(comments), batch_size)]
        results = list(executor.map(lambda batch: summarize_batch(batch, model, tokenizer, cache), batches))

    return [item for sublist in results for item in sublist]

# Adding summary column for a single column 'comments'
def add_summary_column_parallel(df, comment_column='comments', summary_column='summary_comments', batch_size=256):
    model, tokenizer = initialize_model()
    comments = df[comment_column].tolist()

    cache = {}  # Cache for storing summaries

    start_time = time.time()

    summaries = summarize_batch_parallel(comments, model, tokenizer, cache, batch_size=batch_size)

    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"Time taken for processing: {elapsed_time:.2f} seconds")

    # Use .copy() to avoid SettingWithCopyWarning
    df = df.copy()
    df.loc[:, summary_column] = summaries

    # Clear GPU cache to release unnecessary data
    torch.cuda.empty_cache()

    return df

# Example usage
all_comments_summary = add_summary_column_parallel(all_comments_summary, comment_column='comments', summary_column='summary_comments')
print(all_comments_summary['summary_comments'])


Time taken for processing: 9.58 seconds
0            Вот что вы думаете, этот чел просто овощ.
1          Смотри на передние колеса. Подумай над этим
2               Если вы не знаете, что это вообще там?
3    Открой кошельку с рунами. Попробуй доказать, ч...
4    Что вы думаете, что у тебя такой броневик, что...
5       Подарил мне в день рождения стиральную машинку
6    Если вы не хотите выпустить музыку неизданного...
7                              Цена в графитовом цвете
8    Если вы хотите Айон на новом движке, то хотело...
9    Надеюсь, что это не настолько экшен проект за ...
Name: summary_comments, dtype: object


In [30]:
import pandas as pd
from bert_score import score
import numpy as np



# Handle NaN values
all_comments_summary['summary_comments'].fillna("", inplace=True)
all_comments_summary['summary_post'].fillna("", inplace=True)

# Extracting the text from the DataFrame
references = all_comments_summary['summary_comments'].tolist()
hypotheses = all_comments_summary['summary_post'].tolist()

# Calculate BERT_score
P, R, F1 = score(hypotheses, references, lang="ru", rescale_with_baseline=True)

# Create a new DataFrame with BERT_score results
bert_score_df = pd.DataFrame({'Precision': P.tolist(), 'Recall': R.tolist(), 'F1 Score': F1.tolist()})

# Display the BERT_score results DataFrame
bert_score_df


,Precision,Recall,F1 Score
0,0.581713,0.631558,0.605612
1,0.659474,0.646715,0.653032
2,0.634166,0.635067,0.634616
3,0.689978,0.630873,0.659103
4,0.615421,0.643473,0.629134
5,0.905111,0.899318,0.902205
6,0.617681,0.585785,0.601310
7,0.580397,0.619155,0.599150
8,0.594147,0.578265,0.586098
9,0.606290,0.572959,0.589154
